<a href="https://colab.research.google.com/github/wissal9999999999999/-Graph-Based-Enrichment-Ontology-Approach/blob/main/CNNs_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
try:
    import numpy as np
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    os.system("pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113")
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    os.system("pip install -q torchinfo")
    from torchinfo import summary

from pathlib import Path

import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
NUM_WORKERS = os.cpu_count()

[INFO] torch/torchvision versions not as required, installing nightly versions.
torch version: 2.3.0+cu121
torchvision version: 0.18.0+cu121
[INFO] Couldn't find torchinfo... installing it.


In [3]:
import os
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + package.split())
    except subprocess.CalledProcessError as e:
        print(f"[ERROR] Failed to install {package}. Error: {e}")
        sys.exit(1)

try:
    import numpy as np
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except (AssertionError, ImportError) as e:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    print(f"[INFO] Reason: {e}")
    install_package("torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113")
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

try:
    from torchinfo import summary
except ImportError:
    print("[INFO] Couldn't find torchinfo... installing it.")
    install_package("torchinfo")
    from torchinfo import summary

import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from pathlib import Path

# Constants
NUM_WORKERS = os.cpu_count()


[INFO] torch/torchvision versions not as required, installing nightly versions.
[INFO] Reason: torch version should be 1.12+
torch version: 2.3.0+cu121
torchvision version: 0.18.0+cu121


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# training and testing block

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:

    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:


    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }


    model.to(device)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


# Import Data

In [6]:
from google.colab import files
uploaded = files.upload()


Saving datamodel.zip to datamodel.zip


In [7]:
# Extraire le contenu du fichier zip dans le dossier dataset
!unzip -q datamodel.zip -d dataset


In [8]:
import os

# Chemin du dossier dataset
dataset_path = "/content/dataset"

# Noms des classes
classes = os.listdir(os.path.join(dataset_path, 'train'))  # Nous choisissons arbitrairement le dossier 'train' pour obtenir la liste des classes
print("Nombre de classes :", len(classes))

# Parcourir chaque classe
for class_name in classes:
    # Afficher le nom de la classe
    print(f"Classe : {class_name}")

    # Compter le nombre d'images dans chaque ensemble (train, validation, test)
    for subset in ['train', 'validation', 'test']:
        # Chemin du sous-dossier spécifique pour cette classe et cet ensemble
        subset_path = os.path.join(dataset_path, subset, class_name)

        # Vérifier si le dossier existe
        if os.path.exists(subset_path):
            # Compter le nombre d'images dans le sous-dossier
            num_images = len(os.listdir(subset_path))
            print(f"  Nombre d'images dans {subset} : {num_images}")
        else:
            print(f"  Le dossier {subset} pour la classe {class_name} n'existe pas.")


Nombre de classes : 2
Classe : liver_tumor
  Nombre d'images dans train : 12629
  Nombre d'images dans validation : 3563
  Nombre d'images dans test : 1852
Classe : liver_normal
  Nombre d'images dans train : 20520
  Nombre d'images dans validation : 5408
  Nombre d'images dans test : 2937


In [9]:
image_path = "/content/dataset"



# Setup Dirs
train_dir = image_path + "/train"
test_dir = image_path + "/test"


manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])


In [10]:


def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):


  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)


  class_names = train_data.classes


  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names


train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms, # resize, convert images to between 0 & 1 and normalize them
                                                                               batch_size=32) # set mini-batch size to 32


For resnet18

In [11]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm

# Configurer l'appareil
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
weights = torchvision.models.ResNet18_Weights.DEFAULT
model = torchvision.models.resnet18(pretrained=True).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=512,
                    out_features=output_shape,
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best result for resnet18 was 91.25%

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


Epoch 1/10, Train Loss: 0.2652, Train Accuracy: 90.19%, Test Loss: 0.5170, Test Accuracy: 82.15%
Epoch 2/10, Train Loss: 0.1101, Train Accuracy: 95.91%, Test Loss: 0.2006, Test Accuracy: 92.88%
Epoch 3/10, Train Loss: 0.0838, Train Accuracy: 97.03%, Test Loss: 0.5514, Test Accuracy: 83.69%
Epoch 4/10, Train Loss: 0.0563, Train Accuracy: 98.02%, Test Loss: 0.1722, Test Accuracy: 94.32%
Epoch 5/10, Train Loss: 0.0490, Train Accuracy: 98.33%, Test Loss: 0.3892, Test Accuracy: 90.08%
Epoch 6/10, Train Loss: 0.0375, Train Accuracy: 98.74%, Test Loss: 0.2784, Test Accuracy: 89.58%
Epoch 7/10, Train Loss: 0.0329, Train Accuracy: 98.92%, Test Loss: 0.2465, Test Accuracy: 92.15%
Epoch 8/10, Train Loss: 0.0278, Train Accuracy: 99.09%, Test Loss: 0.2681, Test Accuracy: 92.90%
Epoch 9/10, Train Loss: 0.0245, Train Accuracy: 99.27%, Test Loss: 0.2921, Test Accuracy: 93.67%
Epoch 10/10, Train Loss: 0.0220, Train Accuracy: 99.28%, Test Loss: 0.1927, Test Accuracy: 95.07%


In [12]:
def dice_coefficient(preds, labels, smooth=1e-6):
    """
    Compute the Dice coefficient.

    Args:
    preds (torch.Tensor): Predictions from the model (logits).
    labels (torch.Tensor): Ground truth labels.

    Returns:
    float: Dice coefficient score.
    """
    # Apply softmax to predictions if necessary and get predicted classes
    preds = torch.argmax(torch.softmax(preds, dim=1), dim=1)

    # Flatten tensors
    preds = preds.contiguous().view(-1)
    labels = labels.contiguous().view(-1)

    # Compute the intersection
    intersection = (preds * labels).sum().float()
    # Compute the union
    union = preds.sum().float() + labels.sum().float()

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()


In [13]:
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        train_dice = 0  # Initialize train dice score

        for batch in train_dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Précision pour l'ensemble d'entraînement
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Dice coefficient for training
            train_dice += dice_coefficient(outputs, labels)

        train_loss /= len(train_dataloader)
        train_accuracy = 100 * train_correct / train_total


        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0
        test_dice = 0  # Initialize test dice score

        with torch.no_grad():
            for batch in test_dataloader:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                # Précision pour l'ensemble de test
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()



        test_loss /= len(test_dataloader)
        test_accuracy = 100 * test_correct / test_total
        test_dice /= len(test_dataloader)  # Average dice score

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


In [14]:
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=10,
                device=device)

print(results)


NameError: name 'model' is not defined

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

# Définir l'appareil
device = "cuda" if torch.cuda.is_available() else "cpu"

# Charger les poids et le modèle
weights = torchvision.models.ResNet18_Weights.DEFAULT
model = torchvision.models.resnet18(weights=weights).to(device)

# Définir la fonction de perte et l'optimiseur
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Permettre l'entraînement de tous les paramètres du modèle
for param in model.parameters():
    param.requires_grad = True

# Fixer les graines pour la reproductibilité
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Obtenir la forme de sortie (nombre de classes)
output_shape = len(class_names)

# Modifier la couche de classification du modèle
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=model.fc.in_features,
                    out_features=output_shape,
                    bias=True)).to(device)

# Assurez-vous que la fonction train est définie
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for batch in train_dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Précision pour l'ensemble d'entraînement
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss /= len(train_dataloader)
        train_accuracy = 100 * train_correct / train_total

        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for batch in test_dataloader:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                # Précision pour l'ensemble de test
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        test_loss /= len(test_dataloader)
        test_accuracy = 100 * test_correct / test_total

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

# Entraîner le modèle
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=10,
                device=device)

print(results)#best result 95,47


Epoch 1/10, Train Loss: 0.2326, Train Accuracy: 90.50%, Test Loss: 0.3952, Test Accuracy: 83.38%
Epoch 2/10, Train Loss: 0.1093, Train Accuracy: 95.80%, Test Loss: 0.2531, Test Accuracy: 91.84%
Epoch 3/10, Train Loss: 0.0728, Train Accuracy: 97.45%, Test Loss: 0.2729, Test Accuracy: 90.46%
Epoch 4/10, Train Loss: 0.0541, Train Accuracy: 98.10%, Test Loss: 0.1456, Test Accuracy: 94.45%
Epoch 5/10, Train Loss: 0.0466, Train Accuracy: 98.41%, Test Loss: 0.3512, Test Accuracy: 89.37%
Epoch 6/10, Train Loss: 0.0339, Train Accuracy: 98.87%, Test Loss: 0.9188, Test Accuracy: 80.62%
Epoch 7/10, Train Loss: 0.0277, Train Accuracy: 99.10%, Test Loss: 0.3625, Test Accuracy: 90.33%
Epoch 8/10, Train Loss: 0.0218, Train Accuracy: 99.28%, Test Loss: 0.2507, Test Accuracy: 93.51%
Epoch 9/10, Train Loss: 0.0214, Train Accuracy: 99.34%, Test Loss: 0.2685, Test Accuracy: 92.71%
Epoch 10/10, Train Loss: 0.0152, Train Accuracy: 99.54%, Test Loss: 0.1783, Test Accuracy: 95.47%
None


For efficientnet-b0

In [ ]:
def dice_coefficient(preds, labels, smooth=1e-6):
    """
    Compute the Dice coefficient.

    Args:
    preds (torch.Tensor): Predictions from the model (logits).
    labels (torch.Tensor): Ground truth labels.

    Returns:
    float: Dice coefficient score.
    """
    # Apply softmax to predictions if necessary and get predicted classes
    preds = torch.argmax(torch.softmax(preds, dim=1), dim=1)

    # Flatten tensors
    preds = preds.contiguous().view(-1)
    labels = labels.contiguous().view(-1)

    # Compute the intersection
    intersection = (preds * labels).sum().float()
    # Compute the union
    union = preds.sum().float() + labels.sum().float()

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()


In [15]:
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        train_dice = 0  # Initialize train dice score

        for batch in train_dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Précision pour l'ensemble d'entraînement
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Dice coefficient for training
            train_dice += dice_coefficient(outputs, labels)

        train_loss /= len(train_dataloader)
        train_accuracy = 100 * train_correct / train_total
        train_dice /= len(train_dataloader)  # Average dice score

        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0
        test_dice = 0  # Initialize test dice score

        with torch.no_grad():
            for batch in test_dataloader:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                # Précision pour l'ensemble de test
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

                # Dice coefficient for testing
                test_dice += dice_coefficient(outputs, labels)

        test_loss /= len(test_dataloader)
        test_accuracy = 100 * test_correct / test_total
        test_dice /= len(test_dataloader)  # Average dice score

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


In [16]:
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=10,
                device=device)

print(results)


NameError: name 'model' is not defined

In [ ]:
weight = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights
model = torchvision.models.efficientnet_b0(weights=weight).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape,
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best 94.65%

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 68.3MB/s]


Epoch 1/10, Train Loss: 0.1343, Train Accuracy: 94.76%, Test Loss: 0.1970, Test Accuracy: 93.11%
Epoch 2/10, Train Loss: 0.0522, Train Accuracy: 98.23%, Test Loss: 0.2132, Test Accuracy: 94.63%
Epoch 3/10, Train Loss: 0.0378, Train Accuracy: 98.78%, Test Loss: 0.2174, Test Accuracy: 94.57%
Epoch 4/10, Train Loss: 0.0302, Train Accuracy: 98.99%, Test Loss: 0.2174, Test Accuracy: 94.09%
Epoch 5/10, Train Loss: 0.0278, Train Accuracy: 99.08%, Test Loss: 0.1951, Test Accuracy: 93.84%
Epoch 6/10, Train Loss: 0.0236, Train Accuracy: 99.30%, Test Loss: 0.2179, Test Accuracy: 94.13%
Epoch 7/10, Train Loss: 0.0195, Train Accuracy: 99.42%, Test Loss: 0.2101, Test Accuracy: 94.65%
Epoch 8/10, Train Loss: 0.0185, Train Accuracy: 99.43%, Test Loss: 0.2273, Test Accuracy: 93.53%
Epoch 9/10, Train Loss: 0.0191, Train Accuracy: 99.40%, Test Loss: 0.2832, Test Accuracy: 92.09%
Epoch 10/10, Train Loss: 0.0137, Train Accuracy: 99.56%, Test Loss: 0.2688, Test Accuracy: 92.69%


For densenet121

In [ ]:
def dice_coefficient(preds, labels, smooth=1e-6):
    """
    Compute the Dice coefficient.

    Args:
    preds (torch.Tensor): Predictions from the model (logits).
    labels (torch.Tensor): Ground truth labels.

    Returns:
    float: Dice coefficient score.
    """
    # Apply softmax to predictions if necessary and get predicted classes
    preds = torch.argmax(torch.softmax(preds, dim=1), dim=1)

    # Flatten tensors
    preds = preds.contiguous().view(-1)
    labels = labels.contiguous().view(-1)

    # Compute the intersection
    intersection = (preds * labels).sum().float()
    # Compute the union
    union = preds.sum().float() + labels.sum().float()

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()


In [ ]:
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        train_dice = 0  # Initialize train dice score

        for batch in train_dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Précision pour l'ensemble d'entraînement
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Dice coefficient for training
            train_dice += dice_coefficient(outputs, labels)

        train_loss /= len(train_dataloader)
        train_accuracy = 100 * train_correct / train_total
        train_dice /= len(train_dataloader)  # Average dice score

        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0
        test_dice = 0  # Initialize test dice score

        with torch.no_grad():
            for batch in test_dataloader:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                # Précision pour l'ensemble de test
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

                # Dice coefficient for testing
                test_dice += dice_coefficient(outputs, labels)

        test_loss /= len(test_dataloader)
        test_accuracy = 100 * test_correct / test_total
        test_dice /= len(test_dataloader)  # Average dice score

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Train Dice: {train_dice:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%, Test Dice: {test_dice:.4f}")


In [ ]:
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=10,
                device=device)

print(results)


Epoch 1/10, Train Loss: 0.0109, Train Accuracy: 99.68%, Train Dice: 0.9957, Test Loss: 0.1869, Test Accuracy: 95.72%, Test Dice: 0.6331
Epoch 2/10, Train Loss: 0.0051, Train Accuracy: 99.84%, Train Dice: 0.9978, Test Loss: 0.2555, Test Accuracy: 93.26%, Test Dice: 0.6049
Epoch 3/10, Train Loss: 0.0046, Train Accuracy: 99.85%, Train Dice: 0.9979, Test Loss: 0.2754, Test Accuracy: 94.22%, Test Dice: 0.5538
Epoch 4/10, Train Loss: 0.0068, Train Accuracy: 99.79%, Train Dice: 0.9969, Test Loss: 0.2079, Test Accuracy: 95.43%, Test Dice: 0.6816
Epoch 5/10, Train Loss: 0.0032, Train Accuracy: 99.91%, Train Dice: 0.9989, Test Loss: 0.2400, Test Accuracy: 94.11%, Test Dice: 0.5970
Epoch 6/10, Train Loss: 0.0067, Train Accuracy: 99.78%, Train Dice: 0.9971, Test Loss: 0.2319, Test Accuracy: 94.97%, Test Dice: 0.6480
Epoch 7/10, Train Loss: 0.0037, Train Accuracy: 99.88%, Train Dice: 0.9983, Test Loss: 0.2703, Test Accuracy: 93.13%, Test Dice: 0.6370
Epoch 8/10, Train Loss: 0.0063, Train Accuracy: 

In [ ]:
weights = torchvision.models.DenseNet121_Weights.DEFAULT
model = torchvision.models.densenet121(weights=weights).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1024,
                    out_features=output_shape,
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best is 93.46

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 180MB/s]


Epoch 1/10, Train Loss: 0.2353, Train Accuracy: 89.98%, Test Loss: 0.2632, Test Accuracy: 90.21%
Epoch 2/10, Train Loss: 0.1106, Train Accuracy: 95.83%, Test Loss: 0.2275, Test Accuracy: 91.36%
Epoch 3/10, Train Loss: 0.0784, Train Accuracy: 97.10%, Test Loss: 0.2459, Test Accuracy: 91.33%
Epoch 4/10, Train Loss: 0.0554, Train Accuracy: 98.05%, Test Loss: 0.2322, Test Accuracy: 91.54%
Epoch 5/10, Train Loss: 0.0471, Train Accuracy: 98.29%, Test Loss: 0.2209, Test Accuracy: 92.07%
Epoch 6/10, Train Loss: 0.0407, Train Accuracy: 98.65%, Test Loss: 0.4539, Test Accuracy: 87.55%
Epoch 7/10, Train Loss: 0.0330, Train Accuracy: 98.91%, Test Loss: 0.2483, Test Accuracy: 93.46%
Epoch 8/10, Train Loss: 0.0350, Train Accuracy: 98.84%, Test Loss: 0.2114, Test Accuracy: 92.78%
Epoch 9/10, Train Loss: 0.0244, Train Accuracy: 99.19%, Test Loss: 0.2361, Test Accuracy: 92.94%
Epoch 10/10, Train Loss: 0.0210, Train Accuracy: 99.34%, Test Loss: 0.3440, Test Accuracy: 91.81%


In [ ]:
def dice_coef(outputs, labels, smooth=1):
    intersection = (outputs * labels).sum()
    union = outputs.sum() + labels.sum()
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice
weights = torchvision.models.DenseNet121_Weights.DEFAULT
model = torchvision.models.densenet121(weights=weights).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1024,
                    out_features=output_shape,
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best is 93.46

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 127MB/s]


Epoch 1/10, Train Loss: 0.2386, Train Accuracy: 89.94%, Train Dice: 0.8623, Test Loss: 0.3391, Test Accuracy: 85.34%, Test Dice: 0.3593
Epoch 2/10, Train Loss: 0.1180, Train Accuracy: 95.53%, Train Dice: 0.9385, Test Loss: 0.6262, Test Accuracy: 83.80%, Test Dice: 0.3849
Epoch 3/10, Train Loss: 0.0815, Train Accuracy: 97.08%, Train Dice: 0.9601, Test Loss: 0.2350, Test Accuracy: 91.50%, Test Dice: 0.6268
Epoch 4/10, Train Loss: 0.0595, Train Accuracy: 97.91%, Train Dice: 0.9713, Test Loss: 0.3136, Test Accuracy: 88.81%, Test Dice: 0.6172
Epoch 5/10, Train Loss: 0.0499, Train Accuracy: 98.37%, Train Dice: 0.9778, Test Loss: 0.1873, Test Accuracy: 93.26%, Test Dice: 0.6127
Epoch 6/10, Train Loss: 0.0439, Train Accuracy: 98.45%, Train Dice: 0.9789, Test Loss: 0.2364, Test Accuracy: 91.77%, Test Dice: 0.5087
Epoch 7/10, Train Loss: 0.0375, Train Accuracy: 98.77%, Train Dice: 0.9829, Test Loss: 0.2516, Test Accuracy: 91.65%, Test Dice: 0.5331
